In [1]:
!pip install pyspark 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425350 sha256=f25b7996e7be912c4be1bd2a4875e303f9c304a742a1541b4ee5c27aa7ed117d
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import numpy as np
from pyspark.sql.functions import *
import os
import sys
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from functools import reduce
from pyspark.sql import DataFrame
import pyspark
#sc=pyspark.SparkContext()
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext
#import sys
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import lead, lag, col ,datediff, to_date, lit, size , length , year, month, dayofmonth, from_unixtime, unix_timestamp, desc, asc, rank, min , expr, trim, date_format, avg
import datetime
from datetime import datetime
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.types import IntegerType, DoubleType, StringType, DateType, TimestampType, StructField, NumericType
from pyspark.sql.functions import isnan
from pyspark.sql.functions import udf
from io import BytesIO
#import boto3
import math
from pyspark.sql import Row
#import pyarrow as pa
import pandas as pd
from datetime import datetime, timedelta
#from time import gmtime, strftime ,datetime
from pyspark.sql.functions import concat, col, lit
from pyspark.sql.functions import count, min,max
from pyspark.sql.functions import col, when
#import geohash2 as pgh
#import pygeohash as pgh2
spark = SparkSession.builder.appName('NE').getOrCreate()
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/17 08:42:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


DataFrame[key: string, value: string]

In [3]:
df_tele = spark.read.parquet("/kaggle/input/January/January.parquet")

In [4]:
df_region = spark.read.csv("/kaggle/input/network-expansion/all_region_workshops.csv",header=True)

In [5]:
#df_tele.show(3)

In [6]:
#df_region.show(3)
print(df_region.count())

1592


# Python 

In [7]:
import pandas as pd
from openpyxl import load_workbook
from math import radians, cos, sin, asin, sqrt
from tqdm import tqdm
from math import ceil
import numpy as np
import folium
import warnings
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from scipy import stats as st
from sklearn.cluster import KMeans
from statistics import mode
from geopy.geocoders import Nominatim
from sklearn.cluster import DBSCAN
from haversine import haversine
from haversine import Unit

warnings.filterwarnings('ignore')
geolocator = Nominatim(user_agent="geoapiExercises")

In [8]:
def cal_mode(data, latitude, longitude):
    '''
    Returns : the first value if all the values are different
    '''

    # Combining to find mode for Lat & Long both
    data['combined'] = data[latitude] + ' - ' + data[longitude]

    # Calculating mode
    mod = data.groupby(['vehicleId', 'weekofMonth', 'District'])['combined'].apply(lambda x: mode(x)).reset_index()

    for index, val in enumerate(mod['combined']):
        mod.loc[index, 'Lat'] = val.split(' -')[0]
        mod.loc[index, 'Long'] = val.split('- ')[1]

    mod.drop('combined', axis=1, inplace=True)

    return mod

# def haversine(lonlat1, lonlat2):
#     """
#     Calculate the great circle distance between two points
#     on the earth (specified in decimal degrees)
#     """
#     # convert decimal degrees to radians
#     lat1, lon1 = lonlat1
#     lat2, lon2 = lonlat2
#     lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
#
#     # haversine formula
#     dlon = lon2 - lon1
#     dlat = lat2 - lat1
#     a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
#     c = 2 * asin(sqrt(a))
#     r = 6371  # Radius of earth in kilometers. Use 3956 for miles
#     return c * r

def week_of_month(dt):
    """ Returns the week of the month for the specified date."""
    first_day = dt.replace(day=1)

    dom = dt.day
    adjusted_dom = dom + first_day.weekday()

    return int(ceil(adjusted_dom / 7.0))

In [9]:
df_tele.count()

8535257

In [10]:
#df_filtered_region = df_region.filter(col("SSO").isin("Delhi", "Lucknow"))

In [11]:
#df_tele.printSchema()

In [12]:
df_tele = df_tele.withColumn("eventdate",to_date(col("eventdate"),"yyyy-MM-dd"))

In [13]:
#df_tele.show(4)

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, dayofmonth, floor
from pyspark.sql.types import IntegerType
from math import ceil

# Initialize a Spark session
spark = SparkSession.builder.appName("WeekOfMonth").getOrCreate()

# Assuming you have a DataFrame named df_tele
# Convert the "eventdate" column to date type
df_tele = df_tele.withColumn("eventdate", to_date(df_tele["eventdate"], "yyyy-MM-dd"))

# Calculate the day of the month
df_tele = df_tele.withColumn("day_of_month", dayofmonth(df_tele["eventdate"]))

# Define a UDF to calculate the week of the month
from pyspark.sql.functions import udf

def week_of_month(dt):
    """ Returns the week of the month for the specified date."""
    dom = dt.day
    adjusted_dom = dom + (dt.weekday() + 1)  # Adjusted for 1-based indexing
    return int(ceil(adjusted_dom / 7.0))

# Register the UDF
week_udf = udf(week_of_month, IntegerType())

# Calculate the week of the month
df_tele = df_tele.withColumn("week_of_month", week_udf(df_tele["eventdate"]))

# Show the resulting DataFrame
df_tele.show(3)



24/01/17 08:42:23 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+----------+-----------------+-----------+------------+----------+--------------+----------+------------+-------------+
|Unnamed: 0|        vehicleId|gpsLatitude|gpsLongitude| eventdate|odometer_count|  District|day_of_month|week_of_month|
+----------+-----------------+-----------+------------+----------+--------------+----------+------------+-------------+
|         0|MAT422001N2F12927| 28.1071456|  76.8211136|2023-01-01|             3|     Alwar|           1|            2|
|         1|MAT422001N2F13091| 18.2318608|   78.384608|2023-01-01|             2| Kamareddy|           1|            2|
|         2|MAT422001N2F13447| 11.0975896|  77.1377472|2023-01-01|            13|Coimbatore|           1|            2|
+----------+-----------------+-----------+------------+----------+--------------+----------+------------+-------------+
only showing top 3 rows



In [15]:
df_tele.select('day_of_month','week_of_month').show(12)

+------------+-------------+
|day_of_month|week_of_month|
+------------+-------------+
|           1|            2|
|           1|            2|
|           1|            2|
|           1|            2|
|           1|            2|
|           1|            2|
|           1|            2|
|           1|            2|
|           1|            2|
|           1|            2|
|           1|            2|
|           1|            2|
+------------+-------------+
only showing top 12 rows



In [16]:
# Select the distinct values of the "week_of_month" column
distinct_days = df_tele.select("day_of_month").distinct().orderBy("day_of_month")

# Show the distinct values
distinct_days.show()

+------------+
|day_of_month|
+------------+
|           1|
|           2|
|           3|
|           4|
|           5|
|           6|
|           7|
|           8|
|           9|
|          10|
|          11|
|          12|
|          13|
|          14|
|          15|
|          16|
|          17|
|          18|
|          19|
|          20|
+------------+
only showing top 20 rows



In [17]:
distinct_days = df_tele.select("day_of_month").distinct().orderBy("day_of_month").collect()

# Print all the distinct days
for row in distinct_days:
    print(row["day_of_month"])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


In [18]:
#from pyspark.sql.functions import year, month, weekofyear, dayofmonth, dayofweek, dayofyear, hour, minute, second, unix_timestamp
#from pyspark.sql import DataFrame

#def add_datepart_spark(df: DataFrame, field_name: str, prefix=None, drop=True, time=False) -> DataFrame:
    # Extract the date-related features using PySpark functions
    #df = df.withColumn("Year", year(field_name))
    #df = df.withColumn("Month", month(field_name))
    #df = df.withColumn("Week", weekofyear(field_name))
    #df = df.withColumn("Day", dayofmonth(field_name))
    #df = df.withColumn("Dayofweek", dayofweek(field_name))
    #df = df.withColumn("Dayofyear", dayofyear(field_name))
    #df = df.withColumn("Is_month_end", (dayofmonth(field_name) == 31).cast("int"))
    #df = df.withColumn("Is_month_start", (dayofmonth(field_name) == 1).cast("int"))
    #df = df.withColumn("Is_quarter_end", (month(field_name) == 3).cast("int"))
    #df = df.withColumn("Is_quarter_start", (month(field_name) == 1).cast("int"))
    #df = df.withColumn("Is_year_end", (month(field_name) == 12).cast("int"))
    #df = df.withColumn("Is_year_start", (month(field_name) == 1).cast("int"))

    #if time:
        #df = df.withColumn("Hour", hour(field_name))
        #df = df.withColumn("Minute", minute(field_name))
        #df = df.withColumn("Second", second(field_name))

    # Add the "Elapsed" column
    #df = df.withColumn("Elapsed", (unix_timestamp(field_name) * 1000).cast("long"))

    # Drop the original date column if drop is True
    #if drop:
        #df = df.drop(field_name)

    #return df

# Usage:
#df_tele = add_datepart_spark(df_tele, df_tele["eventdate"], prefix='event')


In [19]:
df_tele.printSchema()

root
 |-- Unnamed: 0: long (nullable = true)
 |-- vehicleId: string (nullable = true)
 |-- gpsLatitude: double (nullable = true)
 |-- gpsLongitude: double (nullable = true)
 |-- eventdate: date (nullable = true)
 |-- odometer_count: long (nullable = true)
 |-- District: string (nullable = true)
 |-- day_of_month: integer (nullable = true)
 |-- week_of_month: integer (nullable = true)



In [20]:
#df_tele.show(4)

In [21]:
#df_tele.select("eventdate","day_of_month","week_of_month","Dayofweek","Dayofyear").show(10)

In [22]:
#import pyspark.sql.functions as F

#def cal_mode(data, latitude, longitude):
    # Combine latitude and longitude into a single column
    #data = data.withColumn('combined', F.concat_ws(' - ', data[latitude], data[longitude]))

    # Calculate mode using a custom function
    #mode_udf = F.udf(lambda x: x.mode()[0] if len(x) > 0 else None)
    #mod = data.groupby(['vehicleId', 'week_of_month', 'District']).agg(mode_udf(F.collect_list('combined')).alias('mode'))

    # Split the combined column into 'Lat' and 'Long'
    #mod = mod.withColumn('Lat', F.split(mod['mode'], ' - ')[0])
    #mod = mod.withColumn('Long', F.split(mod['mode'], ' - ')[1])

    # Drop the intermediate 'combined' column
    #mod = mod.drop('mode')

    #return mod

# This is where you define the df_mode variable
#df_mode = cal_mode(df_tele, 'gpsLatitude', 'gpsLongitude')


In [23]:
#print(df_mode.count())
#df_mode.show(3)

In [24]:
import pyspark.sql.functions as F

def cal_mode(data, latitude, longitude):
    # Combining latitude and longitude into a single column
    data = data.withColumn('combined', F.concat_ws(' - ', data[latitude], data[longitude]))
    # Calculating mode using PySpark's aggregation functions
    mod = data.groupby(['vehicleId', 'week_of_month', 'District']) \
              .agg(F.expr('collect_list(combined) as combined_list')) \
              .withColumn('mode', F.expr('aggregate(combined_list, cast(null as string), (acc, x) -> IFNULL(acc, x), acc -> acc)'))
    # Splitting the combined column into 'Lat' and 'Long'
    mod = mod.withColumn('Lat', F.split(mod['mode'], ' - ')[0])
    mod = mod.withColumn('Long', F.split(mod['mode'], ' - ')[1])
    # Drop the intermediate columns
    mod = mod.drop('combined', 'combined_list', 'mode')
    return mod

# This is where you define the df_mode variable
df_mode1 = cal_mode(df_tele, 'gpsLatitude', 'gpsLongitude')


In [25]:
df_mode1.count()

24/01/17 08:43:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:43:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:43:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:43:46 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:43:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:43:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:43:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:43:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:43:55 WARN RowBasedKeyValueBatch: Calling spill() on

4373456

In [26]:
#df_mode1.show()

In [27]:
df_mode1.show(3)

+-----------------+-------------+----------+----------+----------+
|        vehicleId|week_of_month|  District|       Lat|      Long|
+-----------------+-------------+----------+----------+----------+
|MAT422001N2F12927|            2|      Agra|27.1635584|77.8156032|
|MAT422001N2F12927|            3|Chandigarh| 30.700688|76.7895808|
|MAT422001N2F12927|            4|     Alwar|28.1101728|76.8164032|
+-----------------+-------------+----------+----------+----------+
only showing top 3 rows



In [28]:
distinct_vid = df_tele.select("vehicleId").distinct()

# Show the distinct values
distinct_vid.count()

377439

In [29]:

distinct_vid_m = df_mode1.select("vehicleId").distinct()

# Show the distinct values
distinct_vid_m.count()

377439

In [30]:
df_mode1 = df_mode1.withColumn('Lat', col('Lat').cast("float"))

# Cast 'Long' column to Float
df_mode1 = df_mode1.withColumn('Long', col('Long').cast("float"))

# Cast 'District' column to String (assuming it's not already a string)
df_mode1 = df_mode1.withColumn('District', col('District').cast("string"))

In [31]:
df_mode1.columns

['vehicleId', 'week_of_month', 'District', 'Lat', 'Long']

In [32]:
from pyspark.sql.functions import round

# Assuming 'df_mode' is your PySpark DataFrame

# Round 'Lat' column to two decimal places
df_mode1 = df_mode1.withColumn('Lat', round(df_mode1['Lat'], 2))

# Round 'Long' column to two decimal places
df_mode1 = df_mode1.withColumn('Long', round(df_mode1['Long'], 2))


In [33]:
from pyspark.sql import functions as F

# Group by 'District' and count unique combinations of 'Lat' and 'Long'
rou_off = df_mode1.groupBy('District', 'Lat', 'Long').agg(F.count("*").alias("count"))

# Reset the index (not applicable to PySpark, but relevant in Pandas)
# In PySpark, the result will be a DataFrame without an index

# Print the number of rows and columns
row_count = rou_off.count()
column_count = len(rou_off.columns)

print("Number of rows:", row_count)
print("Number of columns:", column_count)


Number of rows: 390570
Number of columns: 4


In [34]:
rou_off.count()

390664

In [35]:
rou_off.columns

['District', 'Lat', 'Long', 'count']

In [36]:
#df_region - has SSO
#rou_off - has Distirct

#df_region_renamed = df_region.withColumnRenamed("SSO", "District")

state_sso_dis = spark.read.csv("/kaggle/input/district/District to SSO master.csv",header=True,inferSchema=True)

In [37]:
df_region.columns

['Division', 'Lattitude', 'Longitude', 'SSO', 'State']

In [38]:
state_sso_dis.columns

['District', 'vehicleId', 'State', 'SSO']

In [39]:
distinct_sso = state_sso_dis.select("SSO").distinct()

In [40]:
distinct_sso.show(32)

+-----------+
|        SSO|
+-----------+
|  Bangalore|
|    Udaipur|
|Bhubaneswar|
|      Jammu|
|      Hubli|
|      Patna|
|    Lucknow|
|    Chennai|
|     Mumbai|
|  Ahmedabad|
|    Kolkata|
|  Allahabad|
|      Salem|
|   Lucknow |
|   Dehradun|
|    Gurgaon|
|       Pune|
|      Delhi|
|        JSR|
|       #N/A|
|     Raipur|
| Chandigarh|
|     Nagpur|
|     Bhopal|
|     Cochin|
|   Siliguri|
|  Hyderabad|
|     Rajkot|
| Vijayawada|
|   Guwahati|
|     Jaipur|
+-----------+



In [41]:
distinct_sso_region = df_region.select("SSO").distinct()

In [42]:
distinct_sso_region.count()

32

In [43]:
distinct_sso.count()

31

In [44]:
state_sso_dis.show(4)

+--------------------+---------+--------------+----------+
|            District|vehicleId|         State|       SSO|
+--------------------+---------+--------------+----------+
|            Adilabad|        3|     Telangana| Hyderabad|
|                Agra|       85| Uttar Pradesh|   Lucknow|
|         Kadapa(YSR)|       40|Andhra Pradesh| Hyderabad|
|Sri Potti Sriramu...|       32|Andhra Pradesh|Vijayawada|
+--------------------+---------+--------------+----------+
only showing top 4 rows



In [45]:
state_sso_dis.count()

635

In [46]:
#df_mode1 = rou_off.join(df_region_renamed, on='District', how='inner')
df_model = rou_off.join(state_sso_dis[['District','SSO','State']], on='District', how='inner')
print(df_mode1.count())

24/01/17 08:50:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:50:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:50:49 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:50:50 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:50:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:50:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:50:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:50:51 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:50:53 WARN RowBasedKeyValueBatch: Calling spill() on

4373456


In [47]:
# Show the resulting DataFrame
df_model.show()

# Print the number of rows and columns
print("Number of rows:", df_mode1.count())
print("Number of columns:", len(df_mode1.columns))

+--------------------+-----+-----+-----+-----------+--------------------+
|            District|  Lat| Long|count|        SSO|               State|
+--------------------+-----+-----+-----+-----------+--------------------+
|              Jaipur|26.82| 75.5|   61|     Jaipur|           Rajasthan|
|               Haora|22.59|88.22| 1010|    Kolkata|         West Bengal|
|          Srikakulam|18.38| 84.0|  217| Vijayawada|      Andhra Pradesh|
|                Durg|21.21|81.29|  264|     Raipur|         Chhatisgarh|
|               Nadia|23.72| 88.3|   12|    Kolkata|         West Bengal|
|              Rohtak|28.89|76.56|  110|    Gurgaon|             Haryana|
|               Morbi|23.07|70.84|   51|     Rajkot|             Gujarat|
|               Patan|23.78|71.62|    5|  Ahmedabad|             Gujarat|
|        Kanpur Nagar|26.45|80.24|  195|    Lucknow|       Uttar Pradesh|
|Dadra & Nagar Haveli|20.31|73.06|   15|  Ahmedabad|Dadra & Nagar Haveli|
|          Jalpaiguri|26.54| 88.8|  22

24/01/17 08:53:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:53:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:54:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:54:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:54:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:54:03 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:54:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:54:04 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/01/17 08:54:05 WARN RowBasedKeyValueBatch: Calling spill() on

Number of rows: 4373456
Number of columns: 5


In [48]:
import pyspark.sql.functions as F

def filter_df(df, sso_col, sso_list):
    '''
    Filters the DataFrame to only the rows where the sso column is in the sso_list
    '''

    # Creating a filter expression
    filter_expr = F.col(sso_col).isin(sso_list)

    # Filtering the DataFrame
    df_filtered = df.filter(filter_expr)

    return df_filtered


In [49]:
sso_list = [row.SSO for row in state_sso_dis.select("SSO").collect()]

In [50]:
# Filtering the DataFrame
final_df = filter_df(df_model, 'SSO', sso_list)

In [51]:
final_df.show()

+--------------------+-----+-----+-----+-----------+--------------------+
|            District|  Lat| Long|count|        SSO|               State|
+--------------------+-----+-----+-----+-----------+--------------------+
|              Jaipur|26.82| 75.5|   62|     Jaipur|           Rajasthan|
|               Patna|25.57|85.24|  337|      Patna|               Bihar|
|               Haora|22.59|88.22| 1004|    Kolkata|         West Bengal|
|          Srikakulam|18.38| 84.0|  219| Vijayawada|      Andhra Pradesh|
|                Durg|21.21|81.29|  265|     Raipur|         Chhatisgarh|
|               Nadia|23.72| 88.3|   12|    Kolkata|         West Bengal|
|              Rohtak|28.89|76.56|  110|    Gurgaon|             Haryana|
|               Morbi|23.07|70.84|   51|     Rajkot|             Gujarat|
|               Patan|23.78|71.62|    5|  Ahmedabad|             Gujarat|
|        Kanpur Nagar|26.45|80.24|  196|    Lucknow|       Uttar Pradesh|
|Dadra & Nagar Haveli|20.31|73.06|   1

In [52]:
#del_cha.count()


In [53]:
final_df.count()

380514

In [54]:
## delhi
work_reg = filter_df(df_region,'SSO',['Delhi', 'Lucknow'])


In [55]:
# PAN-INDIA 
work_reg = filter_df(df_region,'SSO',sso_list)

In [56]:
work_reg.show(4)

+--------------------+---------+----------+------+----------+
|            Division|Lattitude| Longitude|   SSO|     State|
+--------------------+---------+----------+------+----------+
|100B520-Sv&Pa-Nag...|8.1908371|77.4192042| Salem|Tamil Nadu|
|1001650-Sv&Pa-Nag...| 8.198264| 77.431443| Salem|Tamil Nadu|
|2088645-Sv&Pa-Ney...|   8.3681|  77.08208|Cochin|    Kerala|
|2089374-Sv&Pa-Val...| 8.370961| 77.601832| Salem|Tamil Nadu|
+--------------------+---------+----------+------+----------+
only showing top 4 rows



In [57]:
work_reg.count()

1332

In [58]:
final_df.columns

['District', 'Lat', 'Long', 'count', 'SSO', 'State']

In [59]:
work_reg.columns

['Division', 'Lattitude', 'Longitude', 'SSO', 'State']

In [60]:
#del_cha.count()

In [61]:
#del_cha_pd = del_cha.toPandas()
#work_reg_pd = work_reg.toPandas()

In [62]:
final_df.describe().show()


24/01/17 08:58:10 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+--------+------------------+-----------------+------------------+----------+--------------------+
|summary|District|               Lat|             Long|             count|       SSO|               State|
+-------+--------+------------------+-----------------+------------------+----------+--------------------+
|  count|  380499|            380499|           380499|            380499|    380499|              380499|
|   mean|    NULL|22.572518665234348|79.60510274657055|11.108775581538978|      NULL|                NULL|
| stddev|    NULL| 5.602468674327833|5.480422783076958| 50.12034638916672|      NULL|                NULL|
|    min|Adilabad|              8.08|            68.54|                 1|      #N/A|Andaman & Nicobar...|
|    max| Yadadri|             34.61|            97.03|             11872|Vijayawada|         West Bengal|
+-------+--------+------------------+-----------------+------------------+----------+--------------------+



In [63]:
work_reg.describe().show()

+-------+--------------------+------------------+--------------------+----------+--------------+
|summary|            Division|         Lattitude|           Longitude|       SSO|         State|
+-------+--------------------+------------------+--------------------+----------+--------------+
|  count|                1332|              1330|                1330|      1332|          1331|
|   mean|                NULL|22.539841702507598|    705050.736262136|      NULL|          NULL|
| stddev|                NULL|  6.89309862403507|2.1611854444702957E7|      NULL|          NULL|
|    min|1000030-Sv&Pa-Nag...|         10.043523|           20.288639| Ahmedabad|Andhra Pradesh|
|    max|2089696-Sv&Pa-Bag...|          9.993807|           95.556614|Vijayawada|  West Bengal |
+-------+--------------------+------------------+--------------------+----------+--------------+



In [64]:
final_df.printSchema()

root
 |-- District: string (nullable = true)
 |-- Lat: float (nullable = true)
 |-- Long: float (nullable = true)
 |-- count: long (nullable = false)
 |-- SSO: string (nullable = true)
 |-- State: string (nullable = true)



In [65]:
work_reg.printSchema()

root
 |-- Division: string (nullable = true)
 |-- Lattitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- SSO: string (nullable = true)
 |-- State: string (nullable = true)



In [66]:
final_df_cleaned = final_df.na.drop(subset=["SSO"]).filter(col("SSO") != "#N/A")

In [67]:
work_reg_cleaned = work_reg.na.drop(subset=["SSO"]).filter(col("SSO") != "#N/A")

In [68]:
#r = final_df_cleaned.select('SSO').distinct()

In [69]:
#r.show(30)

In [70]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# Define Haversine distance UDF
@udf(returnType=FloatType())
def haversine_distance(lat1, lon1, lat2, lon2):
    from math import radians, sin, cos, sqrt, atan2

    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = 6371.0 * c  # Earth radius in kilometers

    return distance




In [71]:
final_df_cleaned.printSchema()

root
 |-- District: string (nullable = true)
 |-- Lat: float (nullable = true)
 |-- Long: float (nullable = true)
 |-- count: long (nullable = false)
 |-- SSO: string (nullable = true)
 |-- State: string (nullable = true)



In [72]:
work_reg_cleaned.printSchema

<bound method DataFrame.printSchema of DataFrame[Division: string, Lattitude: string, Longitude: string, SSO: string, State: string]>

In [73]:
min_max_values = final_df_cleaned.agg(min("Lat").alias("min_latitude"),
                        max("Lat").alias("max_latitude"),
                        min("Long").alias("min_longitude"),
                        max("Long").alias("max_longitude"))

# Show the result
min_max_values.show()

+------------+------------+-------------+-------------+
|min_latitude|max_latitude|min_longitude|max_longitude|
+------------+------------+-------------+-------------+
|        8.08|       34.61|        68.54|        97.03|
+------------+------------+-------------+-------------+



In [74]:
min_max_values = work_reg_cleaned.agg(min("Lattitude").alias("min_latitude"),
                        max("Lattitude").alias("max_latitude"),
                        min("Longitude").alias("min_longitude"),
                        max("Longitude").alias("max_longitude"))

# Show the result
min_max_values.show()

+------------+------------+-------------+-------------+
|min_latitude|max_latitude|min_longitude|max_longitude|
+------------+------------+-------------+-------------+
|   10.043523|    9.993807|    20.288639|    95.556614|
+------------+------------+-------------+-------------+



In [75]:
work_reg_cleaned.columns

['Division', 'Lattitude', 'Longitude', 'SSO', 'State']

In [76]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType
from math import radians, sin, cos, sqrt, atan2

# Define the Haversine distance function
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return 6371.0 * c  # Radius of Earth in kilometers

# Register the Haversine function as a UDF
haversine_udf = udf(haversine, DoubleType())


In [77]:
# Cross-join the two DataFrames
combined_df = final_df_cleaned.crossJoin(work_reg_cleaned)

# Calculate the distance for each combination of coordinates
combined_df = combined_df.withColumn(
    "distance",
    haversine_udf(
        col("Lat"), col("Long"), col("Lattitude"), col("Longitude")
    )
)

# Find the minimum distance for each row in final_df_cleaned
from pyspark.sql.window import Window
from pyspark.sql import functions as F

window_spec = Window.partitionBy("District").orderBy("distance")

final_df_with_min_distance = combined_df.withColumn(
    "min_distance",
    F.first("distance").over(window_spec)
)

# Select the relevant columns and drop duplicates
final_df_with_min_distance = final_df_with_min_distance.select(
    "District", "Lat", "Long", "count", "SSO", "State", "min_distance"
).distinct()

# Show the resulting DataFrame with the minimum distances
final_df_with_min_distance.show()


AnalysisException: [AMBIGUOUS_REFERENCE] Reference `SSO` is ambiguous, could be: [`SSO`, `SSO`].

In [ ]:
valid_latitude_range = (-90.0, 90.0)
valid_longitude_range = (-180.0, 180.0)

# Filter and clean work_reg_df
work_reg_cleaned = work_reg_cleaned.withColumn("Latitude",when(col("Lattitude").between(valid_latitude_range[0], valid_latitude_range[1]), col("Lattitude")).otherwise(None)).withColumn("Longitude",when(col("Longitude").between(valid_longitude_range[0], valid_longitude_range[1]), col("Longitude")).otherwise(None)).na.drop()

# Filter and clean del_cha_df
final_df_cleaned = final_df_cleaned.withColumn("Lat",when(col("Lat").between(valid_latitude_range[0], valid_latitude_range[1]), col("Lat")).otherwise(None)).withColumn("Long",when(col("Long").between(valid_longitude_range[0], valid_longitude_range[1]), col("Long")).otherwise(None)).na.drop()

In [ ]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType
from math import radians, sin, cos, sqrt, atan2
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# Define the Haversine distance function
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return 6371.0 * c  # Radius of Earth in kilometers

# Register the Haversine function as a UDF
haversine_udf = udf(haversine, DoubleType())

# Alias the columns in the work_reg_cleaned DataFrame to avoid ambiguity
work_reg_cleaned = work_reg_cleaned.withColumnRenamed("SSO", "Work_SSO")
work_reg_cleaned = work_reg_cleaned.withColumnRenamed("State", "Work_State")
# Cross-join the two DataFrames
combined_df = final_df_cleaned.crossJoin(work_reg_cleaned)

# Calculate the distance for each combination of coordinates
combined_df = combined_df.withColumn(
    "distance",
    haversine_udf(
        col("Lat"), col("Long"), col("Lattitude"), col("Longitude")
    )
)

# Find the minimum distance for each row in final_df_cleaned
window_spec = Window.partitionBy("District").orderBy("distance")

final_df_with_min_distance = combined_df.withColumn(
    "min_distance",
    F.first("distance").over(window_spec)
)

# Select the relevant columns and drop duplicates
final_df_with_min_distance = final_df_with_min_distance.select(
    "District", "Lat", "Long", "count", "SSO", "State", "min_distance"
).distinct()

# Show the resulting DataFrame with the minimum distances
#final_df_with_min_distance.show()


In [ ]:
csv_path = "/kaggle/working/your_file.csv"

final_df_with_min_distance.write.csv(csv_path, header=True, mode="overwrite")

In [ ]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DoubleType
from math import radians, sin, cos, sqrt, atan2
from pyspark.sql.window import Window
from pyspark.sql import functions as F

# Define the Haversine distance function
def haversine(lat1, lon1, lat2, lon2):
    # Ensure that the input values are numeric
    if isinstance(lat1, str) or isinstance(lon1, str) or isinstance(lat2, str) or isinstance(lon2, str):
        return None
    
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return 6371.0 * c  # Radius of Earth in kilometers

# Register the Haversine function as a UDF
haversine_udf = udf(haversine, DoubleType())

# Alias the columns in the work_reg_cleaned DataFrame to avoid ambiguity
work_reg_cleaned = work_reg_cleaned.withColumnRenamed("SSO", "Work_SSO")
work_reg_cleaned = work_reg_cleaned.withColumnRenamed("State", "Work_State")

# Ensure that the columns are of numeric type
final_df_cleaned = final_df_cleaned.withColumn("Lat", final_df_cleaned["Lat"].cast(DoubleType()))
final_df_cleaned = final_df_cleaned.withColumn("Long", final_df_cleaned["Long"].cast(DoubleType()))
work_reg_cleaned = work_reg_cleaned.withColumn("Lattitude", work_reg_cleaned["Lattitude"].cast(DoubleType()))
work_reg_cleaned = work_reg_cleaned.withColumn("Longitude", work_reg_cleaned["Longitude"].cast(DoubleType()))

# Cross-join the two DataFrames
combined_df = final_df_cleaned.crossJoin(work_reg_cleaned)

# Calculate the distance for each combination of coordinates
combined_df = combined_df.withColumn(
    "distance",
    haversine_udf(
        col("Lat"), col("Long"), col("Lattitude"), col("Longitude")
    )
)

# Find the minimum distance for each row in final_df_cleaned
window_spec = Window.partitionBy("District").orderBy("distance")

final_df_with_min_distance = combined_df.withColumn(
    "min_distance",
    F.first("distance").over(window_spec)
)

# Select the relevant columns and drop duplicates
final_df_with_min_distance = final_df_with_min_distance.select(
    "District", "Lat", "Long", "count", "SSO", "State", "min_distance"
).distinct()

# Show the resulting DataFrame with the minimum distances
final_df_with_min_distance.show()
